In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 31 mins
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.2
H2O cluster version age:,4 days
H2O cluster name:,H2O_started_from_R_raj_rnt508
H2O cluster total nodes:,1
H2O cluster free memory:,3.540 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [4]:
people=h2o.get_frame('people')

In [5]:
people

id,bloodType,age,healthyEating,activeLifeStyle,income
1,A,31.5161,4,3,34700
2,A,55.0503,3,4,50500
3,O,37.2219,7,3,35500
4,O,59.5018,7,6,57800
5,O,62.202,2,7,53900
6,AB,20.1412,5,9,27500
7,B,42.821,3,6,38200
8,A,59.9437,1,3,52700
9,A,43.9174,5,7,41100
10,A,39.4609,5,7,37500


In [17]:
train,valid,test= people.split_frame(ratios=[0.81,0.09],
                                     destination_frames=['people_train','people_valid','people_test'],seed=123)

In [18]:
print("%d/%d/%d" % (train.nrows,valid.nrows,test.nrows))

803/103/94


In [19]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator


In [24]:
y="income"
ignoreFields=[y,"id"]
x= [i for i in train.names if i not in ignoreFields]
x

['bloodType', 'age', 'healthyEating', 'activeLifeStyle']

In [25]:
ml=H2OGradientBoostingEstimator(model_id='defaults')

In [26]:
ml.train(x,y,train,validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [27]:
ml.mae(train=True)

913.9117299385118

In [28]:
ml.mae(valid=True)

1377.5897213652227

In [29]:
perf=ml.model_performance(test)
perf.mae()

1358.710454301806

In [32]:
#Overfit gbm
ml2=H2OGradientBoostingEstimator(model_id="overfit",ntrees=1000,max_depth=10)

In [33]:
ml2.train(x,y,train,validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [35]:
ml2.mae(train=True)

182.20030111690537

In [36]:
ml2.mae(valid=True)

1544.0480466741922

In [37]:
perf=ml2.model_performance(test)
perf.mae()

1426.0987808298

    seriously overfitting
    Training error have really come down but validation error and test error goes up